### Mount Google Drive

In [1]:
from google.colab import drive
from os.path import join

# Mounting location on runtime for GDrive
ROOT = '/content/drive'

# Mount GDrive on the runtime
drive.mount(ROOT)

# Create and change directory to workspace folder
WORKING_PATH = '/content/drive/My Drive/Github/ml-team1-july2020'
%cd {WORKING_PATH}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Github/ml-team1-july2020


### Import Dependencies

In [14]:
# Add other folders to the system path
import sys
sys.path.append('/content/drive/My Drive/Github/ml-team1-july2020/TagPredictor')
sys.path.append('/content/drive/My Drive/Github/ml-team1-july2020/ManualTagger')

# Import component classes in other folders
#from TagPredictor.classifier import Classifier
#from TagPredictor.classifier_SVM import Classifier_SVM
from TagPredictor.MultilabelClassifier_SVM import MultilabelClassifier_SVM
from TagPredictor.TagPredictor import TagPredictor
from ManualTagger.ManualTagger import ManualTagger

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import ast

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# Set Pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)   # Disable wrapping


ModuleNotFoundError: ignored

### Class Definition

In [11]:
'''
@file       Annotator.ipynb
@date       2020/08/03
@brief      Top level class that defines the annotation tool and active learning algorithm
'''


'''
@brief  NLP classification annotation tool
'''
class Annotator:
    groundTruthDB = None            # Pandas dataframe of all data with ground truth labels
    labeledDB = None                # Pandas dataframe of labeled data
    unlabeledDB = None              # Pandas dataframe of unlabeled data

    tagPredictor = None             # TagPredictor object
    manualTagger = None             # ManualTagger object

    confidenceThreshold = 0.85       # Prediction confidence threshold to determine if a topic should be passed to ManualTagger


    def __init__(self, datafile):
        # Create databases
        self.groundTruthDB, self.labeledDB, self.unlabeledDB = self.createDatabases(datafile)

        # Set up ManualTagger
        self.manualTagger = ManualTagger(self.groundTruthDB)
    

    '''
    @brief      Performs preprocessing and cleaning on a sentence
    @param      text    String that contains the raw sentence
    @return     text    String that contains the cleaned sentence
    '''
    def cleanText(self, text):
        # Function that checks if all characters in a string are ASCII
        def is_ascii(s):
            return all(ord(c) < 128 for c in s)
        
        # Remove URLs
        text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)

        # Replace newline and tab characters with spaces
        text = text.replace('\n', ' ')
        text = text.replace('\t', ' ')

        # Convert all letters to lowercase
        text = text.lower()

        # Split feature string into a list to perform processing on each word
        wordList = text.split()

        # Remove all stop words
        #stop_words = set(stopwords.words('english'))
        #wordList = [word for word in wordList if not word in stop_words]

        # Remove all words to contain non-ASCII characters
        wordList = [word for word in wordList if is_ascii(word)]

        # Remove all leading/training punctuation, except for '$'
        punctuation = '!"#%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        wordList = [word.strip(punctuation) for word in wordList]

        # Reconstruct text
        text = ' '.join(wordList)

        return text


    '''
    @brief      Loads data from CSV files into Pandas dataframes and performs cleanText() on all columns
    @param      datafile        CSV file with all data
    @return     groundTruthDB   Pandas dataframe of all data with ground truth labels
    @return     labeledDB       Pandas dataframe of the labeled data
    @return     unlabeledDB     Pandas dataframe of the unlabeled data
    '''
    def createDatabases(self, datafile):
        # Load CSV file as ground truth database
        groundTruthDB = pd.read_csv(datafile)

        # Combine topic title and leading comment columns
        groundTruthDB['Bag_of_Words'] = groundTruthDB['Topic Title'] + groundTruthDB['Leading Comment']
        groundTruthDB['Bag_of_Words'] = groundTruthDB['Bag_of_Words'].str.strip().str.replace('   ', ' ').str.replace('  ', ' ')

        # Delete unused columns
        groundTruthDB = groundTruthDB.drop(columns=['Topic Title', 'Leading Comment', 'Unnamed: 0'])
        
        # Apply cleanText() to the bag of words
        groundTruthDB['Bag_of_Words'] = groundTruthDB['Bag_of_Words'].apply(lambda x: self.cleanText(x))

        # Code to duplicate multi-tag topics if necessary
        '''
        #create an offset value
        offset = 0
        #the total number of unique comments
        total = len(groundTruthDB)
        for index, entry in enumerate(groundTruthDB['Bag_of_Words']):
            #create a duplicate if post has multiple tags
            tag_list = ast.literal_eval(groundTruthDB.loc[index, 'Tags'])
            text = groundTruthDB.loc[index,'Bag_of_Words']
            while (isinstance(tag_list, list) and len(tag_list) > 1):
                #print(index)
                #sets the tag for the duplicate to a string
                groundTruthDB.loc[total+offset, 'Tags'] = tag_list.pop()
                #Adds the duplicate to the end of the pandas dataframe
                groundTruthDB.loc[total+offset, 'Bag_of_Words'] = text
                offset = offset + 1
            #Changes the first tag to a string
            if (len(tag_list) == 1):
                groundTruthDB.loc[index, 'Tags'] = tag_list.pop()
            #Changes empty tags from lists to strings
            if (isinstance(groundTruthDB.loc[index, 'Tags'], list)):
                groundTruthDB.loc[index, 'Tags'] = ''
                # Not sure why this element is stored as '[]' instead of ''
        '''

        # Filter out topics with no tags
        groundTruthDB = groundTruthDB[groundTruthDB['Tags'].map(len) > 2]

        # Convert Tag column elements from strings to lists
        groundTruthDB['Tags'] = groundTruthDB.Tags.apply(lambda x: x[1:-1].split(','))

        # Take only a subset of the full dataset, if necessary
        #groundTruthDB = groundTruthDB.sample(1000)

        # Split ground truth database into labeled and unlabelled databases
        unlabeledDB, labeledDB = train_test_split(groundTruthDB, test_size=0.2)

        return groundTruthDB, labeledDB, unlabeledDB


    '''
    @brief      Demonstration function to run the entire annotator application
    @param      classifier      Scikit-learn like classifier class to be used
    @param      maxIterations   Maximum times to run the active learning loop
    @return     None
    '''
    def runApplication(self, classifier, maxIterations=8):
        # Create multilabel binarizer for metric calculations
        mlb = MultiLabelBinarizer()

        # Set up dictionary to keep track of statistics
        statDict = {
                    'Active Learning Iteration' :   [],
                    'Labeled Database Size'     :   [],
                    'Unlabeled Database Size'   :   [],
                    'Precision (Tag)'           :   [],
                    'Recall (Tag)'              :   [],
                    'F1 Score (Tag)'            :   [],
                    'Hamming Loss'              :   [],
                    'Accuracy (Tag List)'       :   []}

        # Set up TagPredictor object
        tagPredictor = TagPredictor(classifier, self.labeledDB)

        # Train tagPredictor
        tagPredictor.train()

        # Predict tags for all unlabeled topics
        tagList, confidenceList = tagPredictor.predict(self.unlabeledDB['Bag_of_Words'])

        # Continue running the active learning loop as long as there are still low-confidence topics
        counter = 1
        print('Minimum Confidence:', min(confidenceList))
        print('Maximum Confidence:', max(confidenceList))
        while (any(p < self.confidenceThreshold for p in confidenceList) == True and counter <= maxIterations):
            ## Start of statistic logging
            labeledDBSize = len(self.labeledDB)
            unlabeledDBSize = len(self.unlabeledDB)

            # Calculate Hamming Loss and tag accuracy at the tag list level
            trueLabelIndicatorMatrix = mlb.fit_transform(self.unlabeledDB['Tags'])
            predictedLabelIndicatorMatrix = mlb.transform(tagList)
            hammingLoss = hamming_loss(trueLabelIndicatorMatrix, predictedLabelIndicatorMatrix)
            accuracy = accuracy_score(trueLabelIndicatorMatrix, predictedLabelIndicatorMatrix)

            # Calculate average precision, recall and F1 score at the tag level (not the tag list level)
            precisions = np.zeros(trueLabelIndicatorMatrix.shape[0])
            recalls = np.zeros(trueLabelIndicatorMatrix.shape[0])
            fscores = np.zeros(trueLabelIndicatorMatrix.shape[0])
            for i in range(trueLabelIndicatorMatrix.shape[0]):
                pArray, rArray, fArray, _ = precision_recall_fscore_support(trueLabelIndicatorMatrix[i], predictedLabelIndicatorMatrix[i])
                precisions[i] = pArray[1]
                recalls[i] = rArray[1]
                fscores[i] = fArray[1]
            precision = np.average(precisions)
            recall = np.average(recalls)
            fscore = np.average(fscores)

            # Print out active learning statistics
            print('Active Learning Iteration:', counter)
            print('Labeled Database Size:', labeledDBSize)
            print('Unlabeled Database Size:', unlabeledDBSize)
            print('Precision:', precision)
            print('Recall:', recall)
            print('F1 Score:', fscore)
            print('Hamming Loss:', hammingLoss)
            print('Accuracy:', accuracy)
            
            # Append statistics to statDict
            statDict['Active Learning Iteration'].append(counter)
            statDict['Labeled Database Size'].append(labeledDBSize)
            statDict['Unlabeled Database Size'].append(unlabeledDBSize)
            statDict['Precision (Tag)'].append(precision)
            statDict['Recall (Tag)'].append(recall)
            statDict['F1 Score (Tag)'].append(fscore)
            statDict['Hamming Loss'].append(hammingLoss)
            statDict['Accuracy (Tag List)'].append(accuracy)

            ## End of statistics logging
            
            # Get low-confidence topic indices
            lowConfIndices = [i for i in range(len(confidenceList)) if confidenceList[i] < self.confidenceThreshold]

            # Pass low-confidence topics to the manual tagger
            lowConfTopics = self.unlabeledDB.iloc[lowConfIndices]
            labeledTopics = self.manualTagger.run(lowConfTopics)

            # Add manually tagged topics to the labeled database
            self.labeledDB = pd.concat([self.labeledDB, labeledTopics], join='inner')

            # Remove tagged topics from unlabeled database
            cond = self.unlabeledDB['Bag_of_Words'].isin(lowConfTopics['Bag_of_Words'])
            self.unlabeledDB.drop(self.unlabeledDB[cond].index, inplace=True)

            # Exit active learning loop if there are no more topics in the unlabeled database
            if (len(self.unlabeledDB) == 0):
                break

            # Train tagPredictor with updated labeled database
            tagPredictor = TagPredictor(classifier, self.labeledDB)
            tagPredictor.train()

            # Predict tags for all unlabeled topics
            tagList, confidenceList = tagPredictor.predict(self.unlabeledDB['Bag_of_Words'])

            counter += 1
        
        # Save statistics to a CSV file
        statDataframe =  pd.DataFrame(statDict)
        statDataframe.to_csv('Annotator_Statistics_2080_85_2.csv') 


### Main

In [12]:
# Path to CSV datafile
datafile = '/content/drive/My Drive/Github/ml-team1-july2020/datasets/Team1Dataset.csv'

# Instantiate Annotator object
annotator = Annotator(datafile)

# Run annotation application
annotator.runApplication(MultilabelClassifier_SVM)

Started training
Finished training
Minimum Confidence: 0.6102929099175769
Maximum Confidence: 0.9727306292783482
Active Learning Iteration: 1
Labeled Database Size: 1452
Unlabeled Database Size: 5804
Precision: 0.38100160808637723
Recall: 0.337942223753733
F1 Score: 0.34444635883298874
Hamming Loss: 0.06162858506070084
Accuracy: 0.22191592005513439
Started training


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Finished training
Active Learning Iteration: 2
Labeled Database Size: 4142
Unlabeled Database Size: 3114
Precision: 0.43727253264825516
Recall: 0.3939199314921858
F1 Score: 0.4025797473774352
Hamming Loss: 0.05495034830294946
Accuracy: 0.3015414258188825
Started training


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Finished training
Active Learning Iteration: 3
Labeled Database Size: 4835
Unlabeled Database Size: 2421
Precision: 0.46234338427646976
Recall: 0.4170797191243288
F1 Score: 0.42679333608701636
Hamming Loss: 0.05228290916023258
Accuracy: 0.32837670384138784
Started training


...

Finished training
Active Learning Iteration: 7
Labeled Database Size: 5155
Unlabeled Database Size: 2102
Precision: 0.477243894703457
Recall: 0.4308198541071995
F1 Score: 0.44132572153504596
Hamming Loss: 0.05094049623069604
Accuracy: 0.3434823977164605
Started training


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Finished training
Active Learning Iteration: 8
Labeled Database Size: 5164
Unlabeled Database Size: 2093
Precision: 0.477384933906673
Recall: 0.4321946169772256
F1 Score: 0.44202898550724634
Hamming Loss: 0.050883898709985664
Accuracy: 0.34400382226469184
Started training


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Finished training


### Test Code

In [7]:
# Set up TagPredictor object
tagPredictor = TagPredictor(MultilabelClassifier_SVM, annotator.labeledDB)

# Train tagPredictor
tagPredictor.train()

Started training
[" 'beautifulsoup'" " 'nlp'" " 'nltk'" " 'scrapy'" " 'selenium-webdriver'"
 " 'sentiment-analysis'" " 'splinter'" " 'text-classification'"
 " 'text-mining'" " 'tf-idf'" " 'web-scraping'" " 'word-embedding'"
 "'beautifulsoup'" "'nlp'" "'nltk'" "'scikit-learn'" "'scrapy'"
 "'selenium'" "'selenium-webdriver'" "'sentiment-analysis'" "'splinter'"
 "'text-classification'" "'text-mining'" "'tf-idf'" "'web-scraping'"
 "'word-embedding'"]
Finished training


In [9]:
# Predict tags for all unlabeled topics
tagList, confidenceList = tagPredictor.predict(annotator.unlabeledDB['Bag_of_Words'])
print(tagList)
#print(confidenceList)

[("'scikit-learn'",), ("'text-mining'",), ("'tf-idf'",), ("'text-classification'",), (" 'beautifulsoup'",), ("'tf-idf'", "'word-embedding'"), (" 'beautifulsoup'",), ("'scrapy'",), ("'text-mining'",), (" 'beautifulsoup'",), (" 'beautifulsoup'",), ("'text-mining'",), ("'text-mining'",), ("'scrapy'",), (" 'beautifulsoup'",), ("'web-scraping'",), ("'tf-idf'",), (" 'beautifulsoup'",), (" 'beautifulsoup'", "'web-scraping'"), ("'nltk'",), ("'selenium'",), ("'tf-idf'",), (" 'beautifulsoup'",), (" 'beautifulsoup'",), (" 'beautifulsoup'",), (" 'beautifulsoup'",), ("'scikit-learn'",), (" 'beautifulsoup'",), ("'splinter'",), ("'selenium'",), (" 'beautifulsoup'",), (" 'beautifulsoup'",), (" 'beautifulsoup'",), ("'selenium'",), (" 'beautifulsoup'",), (" 'beautifulsoup'",), ("'text-mining'",), (" 'beautifulsoup'",), (" 'beautifulsoup'",), (" 'beautifulsoup'",), ("'selenium'",), (" 'word-embedding'", "'tf-idf'"), (" 'beautifulsoup'",), (" 'beautifulsoup'",), ("'text-mining'",), (" 'beautifulsoup'",), 